In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

'apple!' : input_data => apple, output_data=pple!

In [15]:
input_str='apple'
label_str='pple!'
char_set=sorted(list(set(input_str+label_str)))
print(char_set)
char_set_size=len(char_set)
print(char_set_size)

['!', 'a', 'e', 'l', 'p']
5


In [16]:
input_size=5
hidden_size=5
output_size=5
learning_rate=0.1

In [17]:
char_to_index=dict((c, i) for i, c in enumerate(char_set))
print(char_to_index)
# char_list=[c for c in char_set]

# char_list=[]
# for c in char_set:
#     char_set.append(c)


{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [18]:
char_to_index.items()
# char_to_index.values()
# char_to_index.keys()

dict_items([('!', 0), ('a', 1), ('e', 2), ('l', 3), ('p', 4)])

In [19]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value]=key

print(index_to_char)


{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [20]:
x_data=[char_to_index[c] for c in input_str] #apple
y_data=[char_to_index[c] for c in label_str] #pple!

x_data=[x_data] #
y_data=[y_data] #

print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [21]:
x_one_hot=[np.eye(char_set_size)[x] for x in x_data]
print(x_one_hot)


[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [22]:
X=torch.FloatTensor(x_one_hot)
Y=torch.LongTensor(y_data)

print(X)
print(Y)

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
tensor([[4, 4, 3, 2, 0]])


C:\Users\it\AppData\Local\Temp\ipykernel_7236\1041949374.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  X=torch.FloatTensor(x_one_hot)


In [23]:
print(X.size(), Y.size())

torch.Size([1, 5, 5]) torch.Size([1, 5])


In [24]:
#nn.RNN(input_size, hidden_size, output_size)
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn=nn.RNN(input_size, hidden_size, batch_first=True)
        # batch_first : 입력데이터의 배치의 차원의 위치를 설정
        # (배치크기, 시퀀스 길이, 특징 수)
        #  입력의 형태가 (32, 10, 50) :  배치의 크기 32, 시퀀스 길이 : 10, 특징의 수 50
        self.fc=nn.Linear(hidden_size,output_size, bias=True)

    def forward(self, x):
        x, _status=self.rnn(x)
        x=self.fc(x)
        return x

In [25]:
model=Net(input_size, hidden_size, output_size)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
output=model(X)
print(output)
print(output.size())


tensor([[[ 0.0215,  0.0040,  0.5907,  0.3384,  0.4782],
         [ 0.3937, -0.0516,  0.2762,  0.7218,  0.3098],
         [ 0.4521,  0.0609,  0.3721,  0.9256,  0.2979],
         [ 0.2608,  0.0413,  0.3342,  0.6207,  0.3506],
         [ 0.6853,  0.0464,  0.3353,  0.8795,  0.1521]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 5, 5])


In [28]:
result=output.data.numpy().argmax(axis=-1) # a[1][5][5] 0:3차원,1:열,2 :행, a[4][4]: 0:열, 1:행
print(np.squeeze(result))  #크기가 1인 차원을 제거, unsqueeze(차원번호):행당차원을 크기1일 차원을 추가 

[2 3 3 3 3]


In [29]:
for i in range(200):
    optimizer.zero_grad()
    output=model(X)
    loss=criterion(output.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result=output.data.numpy().argmax(axis=-1) # 인덱스로된 결과(12111)
    str_result=''.join([index_to_char[c] for c in np.squeeze(result)]) # 문자로된 결과(aeaaa)
    print(i, "loss",loss.item(), "prediction:", result, str_result,'true:',y_data)


0 loss 1.4526150226593018 prediction: [[2 3 3 3 3]] ellll true: [[4, 4, 3, 2, 0]]
1 loss 1.1933889389038086 prediction: [[4 4 3 4 0]] pplp! true: [[4, 4, 3, 2, 0]]
2 loss 0.9512724876403809 prediction: [[4 4 3 4 0]] pplp! true: [[4, 4, 3, 2, 0]]
3 loss 0.728887140750885 prediction: [[4 4 3 4 0]] pplp! true: [[4, 4, 3, 2, 0]]
4 loss 0.5259556174278259 prediction: [[4 4 3 4 0]] pplp! true: [[4, 4, 3, 2, 0]]
5 loss 0.34559518098831177 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
6 loss 0.21672017872333527 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
7 loss 0.13748954236507416 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
8 loss 0.08973627537488937 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
9 loss 0.06016366556286812 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
10 loss 0.04104018583893776 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
11 loss 0.028282901272177696 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
12 loss

In [30]:
sentence = """if you want to build a ship, don't drum up people together to
collect wood and don't assign them tasks and work, but rather
teach them to long for the endless immensity of the sea."""

In [31]:
char_set1=list(set(sentence))
print(char_set1)

[',', '\n', 'y', "'", 'p', 'e', 'w', 'h', 'c', 'a', 's', 'r', 'l', 'k', 'i', 'f', 'u', '.', 'm', 'o', ' ', 'd', 't', 'b', 'n', 'g']


In [32]:
char_dic={c:i for i, c in enumerate(char_set1)}
char_dic

{',': 0,
 '\n': 1,
 'y': 2,
 "'": 3,
 'p': 4,
 'e': 5,
 'w': 6,
 'h': 7,
 'c': 8,
 'a': 9,
 's': 10,
 'r': 11,
 'l': 12,
 'k': 13,
 'i': 14,
 'f': 15,
 'u': 16,
 '.': 17,
 'm': 18,
 'o': 19,
 ' ': 20,
 'd': 21,
 't': 22,
 'b': 23,
 'n': 24,
 'g': 25}

In [33]:
char_dic_size=len(char_dic)
print(char_dic_size)

26


In [34]:
hidden_size=char_dic_size
sequence_length=10  #훈련데이터 batach size
learning_rate=0.1

In [35]:
x_data=[]
y_data=[]

for i in range(0, len(sentence)-sequence_length):
    x_str=sentence[i:i+sequence_length]
    y_str=sentence[i+1:i+sequence_length+1]
    print(i, x_str,':', y_str)
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan : f you want
1 f you want :  you want 
2  you want  : you want t
3 you want t : ou want to
4 ou want to : u want to 
5 u want to  :  want to b
6  want to b : want to bu
7 want to bu : ant to bui
8 ant to bui : nt to buil
9 nt to buil : t to build
10 t to build :  to build 
11  to build  : to build a
12 to build a : o build a 
13 o build a  :  build a s
14  build a s : build a sh
15 build a sh : uild a shi
16 uild a shi : ild a ship
17 ild a ship : ld a ship,
18 ld a ship, : d a ship, 
19 d a ship,  :  a ship, d
20  a ship, d : a ship, do
21 a ship, do :  ship, don
22  ship, don : ship, don'
23 ship, don' : hip, don't
24 hip, don't : ip, don't 
25 ip, don't  : p, don't d
26 p, don't d : , don't dr
27 , don't dr :  don't dru
28  don't dru : don't drum
29 don't drum : on't drum 
30 on't drum  : n't drum u
31 n't drum u : 't drum up
32 't drum up : t drum up 
33 t drum up  :  drum up p
34  drum up p : drum up pe
35 drum up pe : rum up peo
36 rum up peo : um up peop
37 um up pe

In [36]:
print(x_data)
print(y_data)

[[14, 15, 20, 2, 19, 16, 20, 6, 9, 24], [15, 20, 2, 19, 16, 20, 6, 9, 24, 22], [20, 2, 19, 16, 20, 6, 9, 24, 22, 20], [2, 19, 16, 20, 6, 9, 24, 22, 20, 22], [19, 16, 20, 6, 9, 24, 22, 20, 22, 19], [16, 20, 6, 9, 24, 22, 20, 22, 19, 20], [20, 6, 9, 24, 22, 20, 22, 19, 20, 23], [6, 9, 24, 22, 20, 22, 19, 20, 23, 16], [9, 24, 22, 20, 22, 19, 20, 23, 16, 14], [24, 22, 20, 22, 19, 20, 23, 16, 14, 12], [22, 20, 22, 19, 20, 23, 16, 14, 12, 21], [20, 22, 19, 20, 23, 16, 14, 12, 21, 20], [22, 19, 20, 23, 16, 14, 12, 21, 20, 9], [19, 20, 23, 16, 14, 12, 21, 20, 9, 20], [20, 23, 16, 14, 12, 21, 20, 9, 20, 10], [23, 16, 14, 12, 21, 20, 9, 20, 10, 7], [16, 14, 12, 21, 20, 9, 20, 10, 7, 14], [14, 12, 21, 20, 9, 20, 10, 7, 14, 4], [12, 21, 20, 9, 20, 10, 7, 14, 4, 0], [21, 20, 9, 20, 10, 7, 14, 4, 0, 20], [20, 9, 20, 10, 7, 14, 4, 0, 20, 21], [9, 20, 10, 7, 14, 4, 0, 20, 21, 19], [20, 10, 7, 14, 4, 0, 20, 21, 19, 24], [10, 7, 14, 4, 0, 20, 21, 19, 24, 3], [7, 14, 4, 0, 20, 21, 19, 24, 3, 22], [14, 4,

In [37]:
x_one_hot=[np.eye(char_dic_size)[x] for x in x_data]
X=torch.FloatTensor(x_one_hot)
Y=torch.LongTensor(y_data)
print(X)
print(Y)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [39]:
print(Y)

tensor([[15, 20,  2,  ...,  9, 24, 22],
        [20,  2, 19,  ..., 24, 22, 20],
        [ 2, 19, 16,  ..., 22, 20, 22],
        ...,
        [20, 19, 15,  ..., 20, 10,  5],
        [19, 15, 20,  ..., 10,  5,  9],
        [15, 20, 22,  ...,  5,  9, 17]])


In [38]:
print(X.size(), Y.size())

torch.Size([170, 10, 26]) torch.Size([170, 10])


In [40]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn=nn.RNN(input_dim, hidden_dim, num_layers=layers,
                        batch_first=True)
        self.fc=nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status=self.rnn(x)
        x=self.fc(x)
        return x

In [41]:
model=Net(char_dic_size, hidden_size, 2) # 층을 2개 쌓겠다는 의미
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [50]:
outputs=model(X)
print(outputs)

tensor([[[ -7.1442,  -8.6970,  -3.7922,  ...,  -3.7411,   0.4027,   6.4637],
         [-10.8745,   6.4865,  -0.1499,  ..., -11.8867,   3.0097,   0.5563],
         [ -4.0302,  -9.2886,  12.6087,  ...,   4.3321, -13.8219, -14.8877],
         ...,
         [ -2.6902, -17.2299,   0.3226,  ...,   2.7438,  -3.3340, -17.8767],
         [ -0.6138,  -6.7703, -12.2146,  ...,  -3.1912,  14.1658,  -3.0621],
         [ -1.6748,  -7.8139,  -8.6004,  ...,   5.3498,  -6.8045,  10.8438]],

        [[-11.4472,   2.0388,  -1.3171,  ..., -17.3917,   9.2409, -16.4911],
         [ -4.2748,  -9.7922,  12.8184,  ...,   3.9915, -13.5727, -14.8059],
         [ -7.9228,  -1.3701,   6.0005,  ..., -12.3113,   2.9208, -25.5820],
         ...,
         [ -0.9423,  -6.6622, -12.5005,  ...,  -3.5848,  14.1040,  -3.3522],
         [ -2.8020,  -7.2220,  -8.7478,  ...,   5.5487,  -7.1280,   9.2808],
         [ -0.7601,   2.6930,   2.5661,  ...,  -7.6292, -11.5865, -22.7638]],

        [[ -3.9628,  -1.3811,   3.8838,  ...

In [45]:
# result=outputs.view(-1,char_dic_size)
# print(result.size())

torch.Size([1700, 26])


In [46]:
a=['p','y','t','h','o','n']
a1=''.join(a)
a1

'python'

In [51]:
for i in range(200):
    optimizer.zero_grad()
    outputs=model(X)
    loss=criterion(outputs.view(-1,char_dic_size), Y.view(-1))

    loss.backward()
    optimizer.step()

    results=outputs.argmax(dim=2)
    predict_str=""

    for j, result in enumerate(results):
        if j==0 :
            predict_str+=''.join([char_set1[t] for t in result])
        else:
            predict_str+= char_set1[result[-1]]
    print(predict_str)
            

l you want to build a ship, don't drum up people together to
collect wood and don't assign them tasks and work, but rather
teach them to long for the endless immensity of the sea.
t you want to build a ship, don't drum up people together to
collect wood and don't assign them tasks and work, but rather
teach them to long for the endless immensity of the sea.
t you want to build a ship, don't drum up people together to
collect wood and don't assign them tasks and work, but rather
teach them to long for the endless immensity of the sea.
l you want to build a ship, don't drum up people together to
collect wood and don't assign them tasks and work, but rather
teach them to long for the endless immensity of the sea.
f you want to build a ship, don't drum up people together to
collect wood and don't assign them tasks and work, but rather
teach them to long for the endless immensity of the sea.
t you want to build a ship, don't drum up people together to
collect wood and don't assign them task